In [32]:
import numpy as np
import json
from utils import *

item_matrix = np.load('item_matrix.npy', allow_pickle=True )
item_id = np.load('itemid.npy', allow_pickle=True)
wts = np.load('weights.npy', allow_pickle=True)

f = open('anime_id2name.json')

id2name = json.load(f)
f.close

<function TextIOWrapper.close()>

In [1]:
from load import *

In [8]:
np.where(item_id == 5)[0][0]

1

In [33]:
%%time
M = item_matrix @ item_matrix.T

CPU times: user 4.14 s, sys: 1.54 s, total: 5.68 s
Wall time: 1.73 s


In [34]:
import telebot
from telebot import types

ratings = {}
request_user = {}
API_TOKEN = '5677096457:AAHYpRxfaCw9WnJjRybbF-igrxK3XAe3cXo'
bot = telebot.TeleBot(API_TOKEN)

test_ids = [20, 21, 269, 136]


@bot.message_handler(commands=['suggest'])
def echo_message(message):
    name = message.from_user.username
    if name not in ratings:
        bot.send_message(message.chat.id,
                             text=f'Not enough info')
    else:
        p = np.zeros(13686)
        #print(ratings[name])
        for anime_id, score in ratings[name].items():
            p[anime_id] = score
        recs = M @ p
        recs = downvote_seen_items(recs, test_ids)
        for id_ in topn_recommendations(recs):
            bot.send_message(message.chat.id,
                             text=f'Consider {id2name[str(item_id[id_])]}')
        
    
                
@bot.message_handler(commands=['rate'])
def echo_message(message):
    name = message.from_user.username
    if name not in ratings:
        ratings[name] = {}

    keyboard = types.InlineKeyboardMarkup()
    keys = []
    for i in range(1, 11):
        keys.append(types.InlineKeyboardButton(text=f'{i}', callback_data=i))
    keyboard.add(*keys)
    
    anime_id = np.random.randint(13686)
    request_user[name] = anime_id

    bot.send_message(
        message.from_user.id, 
        text=f'Please rate {id2name[str(item_id[anime_id])]}', 
        reply_markup=keyboard
        )
    

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    name = call.from_user.username
    ratings[name][request_user[name]] = call.data
    bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.id,
                           text=f'{id2name[str(item_id[request_user[name]])]}: {call.data}')
    
        

    
@bot.message_handler(content_types=['text'])
def echo_message(message):
    name = message.from_user.username 
    if name in request_user:
        ratings[name][request_user[name]] = float(message.text)      
                
    

In [35]:
bot.polling()